In [3]:
import pandas as pd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score, classification_report, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier



import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [4]:
categorical_columns = []
accuracy_dict = {}
precision_dict = {}
recall_dict = {}
f1_dict = {}
target_column = input('Please Enter target Column: ')

Please Enter target Column: target


In [20]:
d = pd.read_csv('train.csv')
d.head()

unique_id  metric1  metric2  metric6  metric3  metric4  metric5  \
0          1        4        1        2      5.0      5.0      0.0   
1          2        0        1        1      0.0      0.0    100.0   
2          3        0        1        6     37.2    186.0      0.0   
3          4       24        1        1      0.0      0.0    100.0   
4          5        0        1        1      0.0      4.0      0.0   

   binary_var1                            region  \
0            0  84e516108956e6e374941043b5f73e86   
1            0  185a5eb309e9ba14f65e6486f08b3032   
2            1  84e516108956e6e374941043b5f73e86   
3            0  185a5eb309e9ba14f65e6486f08b3032   
4            0  3a2d6295a828fc1ec5084af382b61e35   

                       sourceMedium  ... page4_top page4_exits  visited_page4  \
0  d3d55c19deb05c118cfed68910a1f6bd  ...       0.0          -1              0   
1  3a987a7bb0a840a40d80e301132f138c  ...       0.0          -1              0   
2  d3d55c19deb05c118cfed68910a1f6bd  ...       0.0          -1              0   
3  335cceef7d447b13ae8fc7c0e0a7e826  ...       0.0          -1              0   
4  335cceef7d447b13ae8fc7c0e0a7e826  ...       0.0          -1              0   

   page5_top  page5_exits  visited_page5  page6_top  page6_exits  \
0        0.0           -1              0        0.0           -1   
1        0.0           -1              0        0.0           -1   
2        0.0           -1              0       30.0            0   
3        0.0           -1              0        0.0            1   
4        0.0           -1              0        0.0           -1   

   visited_page6  target  
0              0       0  
1              0       0  
2              1       0  
3              1       0  
4              0       0  

[5 rows x 33 columns]

In [5]:

def load_dataset(path):
    """
    loads csv file into dataframe and returns it.
    """
    data_df = pd.read_csv(path)

    return data_df

def get_shape(df):
    """
    returns shape of dataframe
    """
    return df.shape()

def get_data_type(data_frame, column_name):
    """
    Returns the datatype of the column in a dadtaframe

    Inputs:

    data_frame: Name of data frame
    column_name: Column name in the dataframe you require to know the type.

    Returns:
    data type of the column.
    """

    return data_frame[column_name].dtypes

def preprocess(df):
    """
    Preprocess the dataset: 
        * Removes any special characters, 
        * Removes spaces in beginning or at the end
        * Checks correlation between columns and drops columns if highly correlated
        * Any dataset specific preprocessing goes here
    
    Returns preprocessed dataframe
    """
    
    df.drop(['unique_id','dayHourMinute'], axis=1, inplace=True)
    
    obj_cols=['region', 'sourceMedium', 'device', 'country']
    
    for col in df:
        if col in obj_cols:
            df.drop(col,axis=1,inplace=True)
    
    to_drop = []

    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find index of feature columns with correlation greater than 0.95
    to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]

    #to_drop

    #print(x.shape)
    for col in to_drop:
        df.drop(col,axis=1,inplace=True)
    #print(x.shape)
    for col in df.columns:
        df[col] = df[col].replace('[^A-Za-z0-9\.\-\s]+','', regex = True)
        df[col] = df[col].replace("^\s+|\s+$","",regex=True)

    return df

def get_unique_values(df,col):

    unique_values = len(df[col].unique())

    return unique_values

def get_categorical_columns(df):

    #categorical_columns = []

    for col in df.columns:
        if len(df[col].unique()) < 15:
            categorical_columns.append(col)

    return categorical_columns

def get_null_values(df, col):
    """
    returns number of null values in a column
    """
    return df[col].isnull().sum()

def treat_missing_columns(df):

    dropped_columns = []

    null_percentage = {}


    for col in df:
    #print(data_df[col].where(data_df[col] == -1).count())
        count = df[col].where(df[col] == -1).count()
        if count:
            null_percentage[col] = count/df.shape[0] * 100
            print("{} Column has {} Null Values".format(col,count))
            print("Percentage of Null Value: ",count/df.shape[0] * 100)
            print('-'*10)
    
    for column, percentage in null_percentage.items():
        if percentage > 90:
            dropped_columns.append(column)
            df.drop(column, axis=1, inplace=True)
        else:
            idx = df[df[col] == -1].index
            df.drop(idx, inplace = True)
    idx = df[df['page6_exits'] == -1].index
    #print(idx)
    df.drop(idx, inplace = True)
    
    for i in dropped_columns:
        print("Dropped {} because more than 90% of values are missing.".format(str(i)))
    
    return dropped_columns

def treat_missing_rows(df):

    threshold = int(0.5 * (len(df.columns)))

    return df.dropna(axis = 0, thresh = threshold)

def impute_missing_values(df):
    for col in df.columns:
        if col in categorical_columns:
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            if df[col].dtype == 'int' or df[col].dtype =='float':
                df[col] = df[col].fillna(df[col].median())
            else:
                df[col] = df[col].fillna(df[col].mode()[0])
    return df

def treat_outliers(df):
    for col in df.columns:
        if col not in categorical_columns:
            if df[col].dtype == 'int' or df[col].dtype == 'float':
                percentiles = df[col].quantile([0.01,0.99]).values
                df[col][df[col] <= percentiles[0]] = percentiles[0]
                df[col][df[col] >= percentiles[1]] = percentiles[1]
    return df

def encode_data(df):
    return pd.get_dummies(df)


def summary_table(df):
    a=0
    b=0
    miss_values=0
    outliers = 0
    tbl_df = pd.DataFrame(columns=['Category', 'Count'])
    tbl_df = tbl_df.append({'Category': "Number of Rows" , 'Count': df.shape[0]},  ignore_index=True)
    tbl_df = tbl_df.append({'Category': "Number of Columns" , 'Count': df.shape[1]},  ignore_index=True)
    
    for x in df.columns:
        
        miss_values += len(df[x]) - df[x].count()
        if x not in categorical_columns:
            if df[x].dtype == 'int' or df[x].dtype == 'float':
                percentiles = df[x].quantile([0.01,0.99]).values
                outliers+=  ((df[x] <= percentiles[0]).value_counts().loc[True])
                outliers+=  ((df[x] >= percentiles[1]).value_counts().loc[True])
            
    
    tbl_df = tbl_df.append({'Category': "Number of Categorical Columns" , 'Count': len(categorical_columns)},  ignore_index=True)
    tbl_df = tbl_df.append({'Category': "Number of Continuous Columns" , 'Count': df.shape[1] - len(categorical_columns)},  ignore_index=True)
    tbl_df = tbl_df.append({'Category': "Number of Missing values" , 'Count': miss_values},  ignore_index=True)
    tbl_df = tbl_df.append({'Category': "Number of Outliers" , 'Count': outliers},  ignore_index=True)
    
    return tbl_df

def apply_pca(df):
    scaler = StandardScaler()
    df = scaler.fit_transform(df)

    pca = PCA(0.95)
    df = pca.fit_transform(df)

    print("Number of Principal Components that explained 95% of data: ",pca.n_components_)
    
    return df



In [6]:
def main(df):
    
    df = load_dataset(df)
   
    print('-'*15)
    print("Loaded Dataset")
    time.sleep(2)
    categorical_columns = get_categorical_columns(df)
    print("categorical_columns: ", categorical_columns)
    print('-'*15)
    print('Dataset Summary')
    k = summary_table(df)
    print(k)
    print('-'*15)
    time.sleep(10)
    #df = date_columns(df)
    time.sleep(2)
    df = preprocess(df)
    print('Preprocessed dataset')
    print('-'*15)
    time.sleep(2)
   
    print('-'*15)
    #df = variance_check(df)
    #print('Checked Variance')
    #print('-'*15)
    time.sleep(2)
    df = treat_outliers(df)
    print("Capped and floored outliers")
    print('-'*15)
    time.sleep(2)
    dropped_cols = treat_missing_columns(df)
    #print('Dropped above columns')
    print('-'*15)
    time.sleep(7)
    #df[target_column] = target_c
    df = treat_missing_rows(df)
    
    print('Dropped rows with more than 50% missing data')
    print('-'*15)
    time.sleep(3)
    #target_c = df[target_column]
    #df = df.drop(target_column, axis=1)
    df = impute_missing_values(df)
    print('Imputed Missing Values')
    print('-'*15)
    time.sleep(5)
    #df = variance_check(df)
    time.sleep(3)
    print('Summary after preprocessing')
    e = summary_table(df)
    print(e[:-1])
    print('-'*15)
    target_c = df[target_column]
    df = df.drop(target_column, axis=1)
    df.to_csv('clean_before_encoding.csv')
    df = encode_data(df)
    print('Encoded Data')
    print('-'*15)
    #call pca here
    #df = pd.DataFrame(apply_pca(df))
    df[target_column] = target_c
    df = df.dropna(axis=0)
    #print(df[0])
    
    return df

In [7]:
#dat_df = main('train.csv')
#dat_df.head()

In [8]:
dat_df['target'].value_counts()

NameError: name 'dat_df' is not defined

In [9]:
def model_selection(file):
    

    #target_column = input('Please Enter target Column: ')

    data = main(file)
    print("-"*15)
    
    print("Logistic Model Building...")
    logistic_accuracy,logistic_precision,logistic_recall,logistic_f1 = logistic_regression(data)
    print("Logistic Regression Scores:")
    print("Accuracy: ",logistic_accuracy)
    print("Precision: ",logistic_precision)
    print("Recall: ",logistic_recall)
    print("f1-score: ",logistic_f1)
    print("-"*15)
    time.sleep(4)
    
    print("Decision Tree Model Building...")
    decision_accuracy,decision_precision,decision_recall,decision_f1 = decision_tree(data)
    print("Decision Tree Scores:")
    print("Accuracy: ",decision_accuracy)
    print("Precision: ",decision_precision)
    print("Recall: ",decision_recall)
    print("f1-score: ",decision_f1)
    print("-"*15)
    time.sleep(4)
    
    print("Random Forest Model Building...")
    random_accuracy,random_precision,random_recall,random_f1 = decision_tree(data)
    print("Random Forest Scores:")
    print("Accuracy: ",random_accuracy)
    print("Precision: ",random_precision)
    print("Recall: ",random_recall)
    print("f1-score: ",random_f1)
    print("-"*15)
    time.sleep(4)
    
    print("SVC Model Building...")
    #svc_accuracy,svc_precision,svc_recall,svc_f1 = svc(data)
    print("SVC Model Scores:")
    #print("Accuracy: ",svc_accuracy)
    #print("Precision: ",svc_precision)
    #print("Recall: ",svc_recall)
    #print("f1-score: ",svc_f1)
    print("-"*15)
    time.sleep(4)
    
    print("Naive Bayes Model Building...")
    nb_accuracy,nb_precision,nb_recall,nb_f1 = naive_bayes(data)
    print("Naive Bayes Scores:")
    print("Accuracy: ",nb_accuracy)
    print("Precision: ",nb_precision)
    print("Recall: ",nb_recall)
    print("f1-score: ",nb_f1)
    print("-"*15)
    time.sleep(4)
    
    print("XG Boost Model Building...")
    xgb_accuracy,xgb_precision,xgb_recall,xgb_f1 = xgbm(data)
    print("XG Boost Scores:")
    print("Accuracy: ",xgb_accuracy)
    print("Precision: ",xgb_precision)
    print("Recall: ",xgb_recall)
    print("f1-score: ",xgb_f1)
    print("-"*15)
    time.sleep(4)
    
    print("KNN Model Building...")
    knn_accuracy,knn_precision,knn_recall,knn_f1 = knn(data)
    print("KNN Scores:")
    print("Accuracy: ",knn_accuracy)
    print("Precision: ",knn_precision)
    print("Recall: ",knn_recall)
    print("f1-score: ",knn_f1)
    print("-"*15)
    time.sleep(4)
    
    print("-"*15)
    top_accuracy = sorted(accuracy_dict.items(), key = lambda x : x[1], reverse=True)[0]
    top_f1 = sorted(f1_dict.items(), key = lambda x : x[1], reverse=True)[0]
    
    print("{} gave the highest accuracy score: {}".format(top_accuracy[0],top_accuracy[1]))
    print("{} gave the highest f1-score: {}".format(top_f1[0],top_f1[1]))
    #print(accuracy_dict)
    #print(precision_dict)
    #print(recall_dict)
    #print(f1_dict)
    

In [19]:
model_selection('train.csv')

---------------
Loaded Dataset
categorical_columns:  ['binary_var1', 'device', 'binary_var2', 'page1_exits', 'visited_page1', 'page2_exits', 'visited_page2', 'page3_exits', 'visited_page3', 'page4_exits', 'visited_page4', 'page5_exits', 'visited_page5', 'page6_exits', 'visited_page6', 'target', 'binary_var1', 'device', 'binary_var2', 'page1_exits', 'visited_page1', 'page2_exits', 'visited_page2', 'page3_exits', 'visited_page3', 'page4_exits', 'visited_page4', 'page5_exits', 'visited_page5', 'page6_exits', 'visited_page6', 'target']
---------------
Dataset Summary
                        Category    Count
0                 Number of Rows   347841
1              Number of Columns       33
2  Number of Categorical Columns       32
3   Number of Continuous Columns        1
4       Number of Missing values        0
5             Number of Outliers  4108659
---------------
Preprocessed dataset
---------------
---------------
Capped and floored outliers
---------------
page1_exits Column has 

In [75]:
top_accuracy = sorted(accuracy_dict.items(), key = lambda x : x[1], reverse=True)[0]
print("{} gave the highest accuracy score: {}".format(top_accuracy[0],top_accuracy[1]))

XG Boost gave the highest accuracy score: 0.9097968163582244


In [11]:
def split_scale_data(df):
    """
    Splits the data into train and test sets and also applies standard 
    scaling to the features
    
    Returns scaled features and target
    
    """
    x = df.drop(target_column,axis=1)
    y = df[target_column]
    
    #from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.15)
    
    #print(x_train.shape)
    #print(x_test.shape)
    #print(y_train.shape)
    #print(y_test.shape)
    
    return x_train, x_test, y_train, y_test

In [12]:

def logistic_regression(df):
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    model = LogisticRegression()

    model.fit(x_train,y_train)

    test_pred = model.predict(x_test)

    accuracy = accuracy_score(y_test, test_pred)
    precision = precision_score(y_test, test_pred)
    recall = recall_score(y_test, test_pred)
    f1 = f1_score(y_test,test_pred)
    
    accuracy_dict['logistic'] = accuracy
    precision_dict['logistic'] = precision
    recall_dict['logistic'] = recall
    f1_dict['logistic'] = f1
    
    return accuracy,precision,recall,f1


In [17]:
logistic_regression(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.9012553384237091,
 0.6025848142164781,
 0.29335430593786865,
 0.3946046019571542)

In [52]:
accuracy_score

0.9060006039428843

In [53]:
f1_sc

0.4162871684971872

In [13]:
def decision_tree(df):
    

    decision_tree_clf = DecisionTreeClassifier(min_samples_split=3, random_state= 31)
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    decision_tree_clf.fit(x_train,y_train)

    test_pred = decision_tree_clf.predict(x_test)

    accuracy = accuracy_score(y_test, test_pred)
    precision = precision_score(y_test, test_pred)
    recall = recall_score(y_test, test_pred)
    f1 = f1_score(y_test, test_pred)
    
    accuracy_dict['decision tree'] = accuracy
    precision_dict['decision tree'] = precision
    recall_dict['decision tree'] = recall
    f1_dict['decision tree'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1

In [30]:
decision_tree(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.877399594495492,
 0.44692737430167595,
 0.4007707129094412,
 0.42259244209670865)

In [14]:
def random_forest(df):
    

    random_model = RandomForestClassifier(n_estimators= 440, random_state= 0)
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    random_model.fit(x_train,y_train)

    preds = random_model.predict(x_test)
    
    accuracy = accuracy_score(y_test, preds)
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    
    accuracy_dict['random forest'] = accuracy
    precision_dict['random forest'] = precision
    recall_dict['random forest'] = recall
    f1_dict['random forest'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1


In [32]:
random_forest(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.8993140934385919,
 0.5689851767388826,
 0.38742236024844723,
 0.46096997690531183)

In [15]:
def svc(df):
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    

    svc_model = SVC(verbose=True)

    svc_model.fit(x_train,y_train)

    svc_preds = svc_model.predict(x_test)
    
    accuracy = accuracy_score(y_test, svc_preds)
    precision = precision_score(y_test, svc_preds)
    recall = recall_score(y_test, svc_preds)
    f1 = f1_score(y_test, svc_preds)
    
    accuracy_dict['svc'] = accuracy
    precision_dict['svc'] = precision
    recall_dict['svc'] = recall
    f1_dict['svc'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1
    

In [34]:
svc(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)
[LibSVM]

(0.9024200854147794, 0.6751173708920187, 0.272865275142315, 0.3886486486486487)

In [16]:
def xgbm(df):
    
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
     
    #import xgboost as xgb
    xgb_model = XGBClassifier()
    xgb_model.fit(x_train, y_train)
    # make predictions for test data
    xgb_model_pred = xgb_model.predict(x_test)
    
    accuracy = accuracy_score(y_test, xgb_model_pred)
    precision = precision_score(y_test, xgb_model_pred)
    recall = recall_score(y_test, xgb_model_pred)
    f1 = f1_score(y_test, xgb_model_pred)
    
    accuracy_dict['XG Boost'] = accuracy
    precision_dict['XG Boost'] = precision
    recall_dict['XG Boost'] = recall
    f1_dict['XG Boost'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1

In [36]:
xgbm(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.905784910055649, 0.6702586206896551, 0.3510158013544018, 0.4607407407407407)

In [17]:
def naive_bayes(df):
    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    

    classifier = GaussianNB()
    classifier.fit(x_train, y_train)
    # Predicting the Test set results
    y_pred = classifier.predict(x_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    accuracy_dict['naive bayes'] = accuracy
    precision_dict['naive bayes'] = precision
    recall_dict['naive bayes'] = recall
    f1_dict['naive bayes'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1
    


In [38]:
naive_bayes(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.8486260299383116,
 0.3538600180234305,
 0.4645110410094637,
 0.4017050298380222)

In [18]:
def knn(df):

    
    x_train, x_test, y_train, y_test = split_scale_data(df)
    
    knn_model = KNeighborsClassifier(n_neighbors=2)

    knn_model.fit(x_train,y_train)

    knn_preds = knn_model.predict(x_test)
    
    accuracy = accuracy_score(y_test, knn_preds)
    precision = precision_score(y_test, knn_preds)
    recall = recall_score(y_test, knn_preds)
    f1 = f1_score(y_test, knn_preds)
    
    accuracy_dict['knn'] = accuracy
    precision_dict['knn'] = precision
    recall_dict['knn'] = recall
    f1_dict['knn'] = f1
    
    #print('Accuracy Score: ', accuracy)
    #print('Precision Score: ', precision)
    #print('Recall Score: ', recall)
    #print('f1-Score: ', f1)
    #print('-'*15)
    
    return accuracy, precision, recall, f1

In [40]:
knn(dat_df)

(131353, 15)
(23181, 15)
(131353,)
(23181,)


(0.8930589707087702, 0.52, 0.20643112346169115, 0.29553850525717534)